In [2]:
import openai,os,requests,json,re
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
# 指定した場所の天気情報を返すダミー関数 (本番では外部APIを利用)
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation():
    # STEP1: モデルにユーザー入力と関数の情報を送る
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "ボストンの天気はどうなんだろう？"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "指定した場所の現在の天気を取得",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都市と州（例：San Francisco, CA)",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    message = response["choices"][0]["message"]
    print("message>>>\n", message, "\n\n")

    # STEP2: モデルが関数を呼び出したいかどうかを確認
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # STEP3: 関数を呼び出す
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        # STEP4: モデルにユーザー入力と関数の実行結果を送る
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "ボストンの天気はどうなんだろう？"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

# 実行
print("response>>>\n", run_conversation()["choices"][0]["message"]["content"], "\n\n")

message>>>
 {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\",\n  \"unit\": \"celsius\"\n}"
  }
} 


response>>>
 ボストンの現在の天気は晴れで、気温は摂氏72度です。風も強いようです。 




In [11]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "2023年7月27日〜29日。竹芝ポートシティでSKS Japan 2023が開催。主催はシグマクシス"}],
    functions=[
        {
            "name": "get_event",
            "description": "イベント情報をJson形式で返す",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string", "description": "イベント名"
                    },
                    "organizer": {
                        "type": "string", "description": "主催者"
                    },
                    "date": {
                        "type": "string", "description": "日付。形式はYYYY/MM/DD~YYYY/MM/DD"
                    },
                    "location": {
                        "type": "string", "description": "開催場所",
                    },
                },
                "required": ["title","organizer","date","location"],
            },
        }
    ],
    function_call="auto",
)
message = response["choices"][0]["message"]
print(json.dumps(json.loads(message['function_call']['arguments']),indent=2,ensure_ascii=False))

{
  "title": "SKS Japan 2023",
  "organizer": "シグマクシス",
  "date": "2023/07/27~2023/07/29",
  "location": "竹芝ポートシティ"
}
